## Decorators
Decorators are a significant part of Python. Simply put, they are functions that modify the functionality of other functions. They help to make our code shorter and more Pythonic

In [1]:
def first_decorator(func):
    def wrap():
        print('before')
        func()
        print('after')
    return wrap

In [2]:
def test():
    """test function docs"""
    print('inside test')

In [3]:
test.__name__

'test'

In [4]:
help(test)

Help on function test in module __main__:

test()
    test function docs



In [5]:
decorated_func = first_decorator(test)

In [6]:
test()

inside test


In [10]:
decorated_func()

before
inside test
after


Instead of writing code like this `decorated_func = first_decorator(test)` we can use special python syntax with `@` symbol

In [8]:
@first_decorator
def test():
    """test function docs"""
    print('inside test')

In [9]:
test()

before
inside test
after


In [10]:
test.__name__ #  the overwrites and docstring of our function

'wrap'

In [12]:
test.__doc__

In [14]:
from functools import wraps

In [15]:
def first_decorator(func):
    @wraps(func) # не обов'язково до використання
    def wrap():
        print('before')
        func()
        print('after')
    return wrap

@first_decorator
def test():
    """test function docs"""
    print('inside test')

In [16]:
test()

before
inside test
after


In [17]:
test.__name__

'test'

In [18]:
test.__doc__

'test function docs'

Passing arguments and a return value

In [20]:
def logging_decorator(func):
    def wrapper(*args, **kwargs):
        print(f"\t[LOG] \t calling {func.__name__}")
        res = func(*args, **kwargs)
        print(f"\t[LOG] called {func.__name__}")
        return res
    return wrapper

# logging_decorator(func)(args)

In [21]:
@logging_decorator
def multiply(number_a:int, number_b:int):
    """ multiplies two provided numbers """
    print(f"\t\t=== Inside the multiply function")
    return number_a * number_b

In [22]:
result = multiply(4, 5)
print(f"[result] \t {result}")

	[LOG] 	 calling multiply
		=== Inside the multiply function
	[LOG] called multiply
[result] 	 20


Multiple decorators

In [23]:
def logging_decorator_1(func):
  def wrapper(*args, **kwargs):
    print(f"[LOG1] \t calling decorator 1")
    res = func(*args, **kwargs)
    print(f"[LOG1] called decorator 1")
    return res
  return wrapper

def logging_decorator_2(func):
  def wrapper(*args, **kwargs):
    print(f"[LOG2] \t calling decorator 2")
    res = func(*args, **kwargs)
    print(f"[LOG2] called decorator 2")
    return res
  return wrapper

In [24]:
@logging_decorator_1
@logging_decorator_2
def multiply(number_a:int, number_b:int):
  """ multiplies two provided numbers """
  print("\tmultiply function")
  return number_a * number_b

In [28]:
result = multiply(4, 5)
print(f"[result] \t {result}")

# def decorator(func):
#     def modyficator(parameters):
#         result = func(parameters)
#         return result
#     return modyficator

[LOG1] 	 calling decorator 1
[LOG2] 	 calling decorator 2
	multiply function
[LOG2] called decorator 2
[LOG1] called decorator 1
[result] 	 20


Passing arguments to decorators

In [26]:
def add_brake_log(size=2):
    def add_brake_log_dec(func):
        @wraps(func)
        def wrap(*args, **kwargs):
            for _ in range(size):
                print('_' * 80)
            func(*args, **kwargs)
            for _ in range(size):
                print('_' * 80)
        return wrap
    return add_brake_log_dec

# def decorator(decorator_parameter=default):
#     def func_getter(func):
#         def modyficator(parameters):
#             if decorator_parameter is one:
#                 func(parameters)
#                 and one functinality
#             elif decorator_parameter is second:
#                 func(parameters)
#                 and another functinality
#             return result
#         return modyficator
#     return func_getter

In [30]:
@add_brake_log(size=3)
def test():
    """test function docs"""
    print('inside test')

test()

________________________________________________________________________________
________________________________________________________________________________
________________________________________________________________________________
inside test
________________________________________________________________________________
________________________________________________________________________________
________________________________________________________________________________


Decorator classes that have state

In [32]:
class ExecutionCounter:

  def __init__(self, func):
    self.func = func
    self.call_count = 0
  def __call__(self, *args, **kwargs):
    self.call_count += 1
    print(f"Called {self.func.__name__} for the {self.call_count}th time")
    return self.func(*args, **kwargs)

In [33]:
@ExecutionCounter
def multiply(number_a:int, number_b:int):
  """ multiplies two provided numbers """
  return number_a * number_b

In [34]:
res1 = multiply(21, 2)
res2 = multiply(14, 5)
res3 = multiply(6, 9)
res4 = multiply(4, 2)
print(f"res: {res1, res2, res3, res4}")

Called multiply for the 1th time
Called multiply for the 2th time
Called multiply for the 3th time
Called multiply for the 4th time
res: (42, 70, 54, 8)


Interaction between decorator args and function args

In [1]:
def check_password(password):
  def outer_wrapper(func):
    def wrapper(*args, **kwargs):
      provided_pass = kwargs.get("password") or args[0]
      if provided_pass != password:
        print(provided_pass, password)
        raise ValueError("Incorrect password")

      res = func(*args, **kwargs)
      return res
    return wrapper
  return outer_wrapper

In [2]:
@check_password(password="mypassword")
def public_api_request(password):
  """ Prints some text if the provided password is correct """
  print("You can only see this if you provided the right answer")

In [3]:
try:
    public_api_request(password='mypassword')
except Exception as e:
    print(e)

try:
    public_api_request(password='wrongpassword')
except Exception as e:
    print(e)

You can only see this if you provided the right answer
wrongpassword mypassword
Incorrect password


Decorators in API
- Валідація JSON
- Кешування
- Задання шляху і методу запиту
- Модифікація параметрів запиту

In [4]:
from functools import lru_cache
@lru_cache(maxsize=4)
def fibonacci(num):
    print(f"Calculating fibonacci({num})")
    if num < 2:
        return num
    return fibonacci(num - 1) + fibonacci(num - 2)

fibonacci(6)
fibonacci(6)
fibonacci(6)
fibonacci(6)
fibonacci(6)

Calculating fibonacci(6)
Calculating fibonacci(5)
Calculating fibonacci(4)
Calculating fibonacci(3)
Calculating fibonacci(2)
Calculating fibonacci(1)
Calculating fibonacci(0)


8

In [40]:
@app.route("/grade", methods=["POST"])
def update_grade():
    json_data = request.get_json()
    if "student_id" not in json_data:
        abort(400)
    # Update database
    return "success!"

NameError: name 'app' is not defined

ImportError: cannot import name 'do_twice' from 'decorators' (/Users/seva/Develop/Beetroot/.venv/lib/python3.11/site-packages/decorators/__init__.py)